In [3]:
from google.colab import files
files.upload()
#Upload your kaggle.json api key

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"liamlio","key":"32d0c187f3ae18696e9889dd18b5baf3"}'}

In [0]:
#Moving the kaggle.json file into the .kaggle/ directory
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c ieee-fraud-detection 
#Download the dataset from the kaggle api

train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
#unzip all the dataset (this will take a minute to appear in the /content directory)
import zipfile
with zipfile.ZipFile("test_identity.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile("test_transaction.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile("train_identity.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile("train_transaction.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')

In [0]:
#Dependencies
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing

In [0]:
#Putting our data into dataframes
test_id = pd.read_csv('test_identity.csv')
test_tr = pd.read_csv('test_transaction.csv')
train_id = pd.read_csv('train_identity.csv')
train_tr = pd.read_csv('train_transaction.csv')

So the transaction dataframe is split into multiple id's where id's 1-11, are continuous data and the rest are categorical.

In [0]:
#Let's merge the datasets for convenience
train = pd.merge(train_tr, train_id, on='TransactionID', how='left')
test = pd.merge(test_tr, test_id, on='TransactionID', how='left')


An extensive amount of data exploratory analysis has already been done and can be found in the public notebooks. 
I'll basing my work off these: [One](https://www.kaggle.com/jesucristo/fraud-complete-eda), [Two](https://www.kaggle.com/artgor/eda-and-models)

In [0]:
train.head()

In [0]:
#From Two we can see that some columns are fairly sparse. Therefore as an initial test
#I'll replace all Nan's with 0.0 and one hot embed all categorical data as a quick start
#to get an idea what data is useful.
#Preprocessing
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9] # if 90% is null in a column
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

In [0]:
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9] #If the columns is over 90% one value
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [0]:
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1] #If the column only has one value
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]

In [12]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
cols_to_drop.remove('isFraud')
len(cols_to_drop)

82

In [0]:
train.drop(cols_to_drop, axis=1)
test.drop(cols_to_drop, axis=1)

In [0]:
cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
for col in cat_cols:
  if col in train.columns:
    le = preprocessing.LabelEncoder()
    le.fit( list(train[col].astype(str).values) +list(test[col].astype(str).values))

    train[col] = le.transform(list(train[col].astype(str).values))
    test[col] = le.transform(list(test[col].astype(str).values))

In [16]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,4,4248,501,50,1,42,1,215,80,19.0,NaN,32,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
1,2987001,0,86401,29.0,4,9979,304,50,2,2,1,225,80,NaN,NaN,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2,2987002,0,86469,59.0,4,11850,390,50,4,66,2,230,80,287.0,NaN,36,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3,2987003,0,86499,50.0,4,8796,467,50,2,17,2,376,80,NaN,NaN,55,32,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
4,2987004,0,86506,50.0,1,11687,414,50,2,2,1,320,80,NaN,NaN,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1,55,7,1,1,65,19,438,44,734,35,3,17,440,115,2,1,1,7,162,3,268,3,1,0,1,1,1,1565


In [0]:
#Splitting the data
X = train.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID', 'isFraud'], axis=1)

y = train.sort_values('TransactionDT')['isFraud']

X_test = test.drop(['TransactionID', 'TransactionDT'], axis=1)

test = test[['TransactionID', 'TransactionDT']]

In [0]:
# by https://www.kaggle.com/dimartinot
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
X = clean_inf_nan(X)
X_test = clean_inf_nan(X_test )

In [17]:
X.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,68.5,4,4248,501,50,1,42,1,215,80,19.0,NaN,32,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
1,29.0,4,9979,304,50,2,2,1,225,80,NaN,NaN,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
2,59.0,4,11850,390,50,4,66,2,230,80,287.0,NaN,36,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
3,50.0,4,8796,467,50,2,17,2,376,80,NaN,NaN,55,32,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,55,28,3,2,127,19,568,547,734,35,3,17,440,115,2,2,2,86,136,6,461,4,2,2,2,2,2,2740
4,50.0,1,11687,414,50,2,2,1,320,80,NaN,NaN,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1,55,7,1,1,65,19,438,44,734,35,3,17,440,115,2,1,1,7,162,3,268,3,1,0,1,1,1,1565


In [0]:
#The next two cells will make a random seaborn graph and linearly model the problem with logistic regression
#This is to complete a homework for the Make Money with Machine Learning course
#I will create a more complicated model after

sns.relplot(data=X, x='id_01', y='TransactionAmt', hue='id_28')

In [38]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X.fillna(0),y)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
clf.predict(X.fillna(0))
clf.score(X.fillna(0),y)

0.9654621194161276

In [0]:
import zipfile
with zipfile.ZipFile("sample_submission.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')

In [0]:
sub = pd.read_csv('sample_submission.csv')
sub['isFraud'] = clf.predict(X_test.fillna(0))

In [0]:
sub.to_csv('submission.csv',index=False)
#This submission got a score of 0.54 (Area under ROC curve)


In [46]:
sub.shape

(506691, 2)